In [7]:
import pandas as pd
import wikipediaapi
import re
import pymorphy2
import numpy as np

In [8]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    words = re.findall(r'\b\w+\b', text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmas)

In [14]:
type_keywords = {
    'исторический': r'культура|культурный|наследие|наследственный|история|исторический|археология|археологический|музей|музейный|крепость|крепостной|древность|древний|памятник|памятный|архив|архивный|летопись|летописный|реставрация|реставрационный|экспонат|экспонатный|хроника|хроникальный|артефакт|артефактный',
    'курортный': r'курорт|курортный|санаторий|санаторный|здравница|здравичный|грязь|грязевой|минеральный|минеральный|пляж|пляжный|термальный|термальный|отель|отельный|туризм|туристический|море|морской|отдых|отдыхный|лечение|лечебный|релакс|релаксационный|оздоровление|оздоровительный',
    'лыжный': r'лыжи|лыжный|подъемник|подъемный|сноуборд|сноубордный|трасса|трассовый|каток|катковый|зима|зимний|снег|снежный|спуск|спусковой|горка|горковый|обледенение|обледенелый|бугель|бугельный|фристайл|фристайловый|сноупарк|сноупарковый',
    'промышленный': r'завод|заводской|фабрика|фабричный|цех|цеховой|индустрия|индустриальный|промзона|промзоновый|технопарк|технопарковый|добыча|добычный|переработка|перерабатывающий|склад|складской|логистика|логистический|производство|производственный|конвейер|конвейерный|сырье|сырьевой',
    'религиозный': r'храм|храмовый|монастырь|монастырский|паломничество|паломнический|святыня|святынный|икона|иконный|богослужение|богослужебный|реликвия|реликвийный|молитва|молитвенный|вера|верный|обряд|обрядовый|священник|священнический|алтарь|алтарный|исповедь|исповедальный',
    'студенческий': r'университет|университетский|академия|академический|студент|студенческий|вуз|вузовский|образование|образовательный|наука|научный|лаборатория|лабораторный|библиотека|библиотечный|лекция|лекционный|кампус|кампусный|исследование|исследовательский|сессия|сессионный|стипендия|стипендиальный',
    'туристический': r'достопримечательность|достопримечательный|экскурсия|экскурсионный|гид|гидовый|путеводитель|путеводительный|отель|отельный|турист|туристический|сувенир|сувенирный|музей|музейный|памятник|памятный|архитектура|архитектурный|путешествие|путешественный|маршрут|маршрутный|фотография|фотографический',
    'торговый': r'рынок|рыночный|магазин|магазинный|торговля|торговый|базар|базарный|бутик|бутиковый|ярмарка|ярмарочный|розница|розничный|опт|оптовый|продажа|продажный|скидка|скидочный|коммерция|коммерческий|акция|акционный|реклама|рекламный',
    'военный': r'гарнизон|гарнизонный|казарма|казарменный|полигон|полигонный|армия|армейский|флот|флотский|оборона|оборонительный|база|базовый|укрепление|укрепительный|стратегия|стратегический|ветеран|ветеранский|парад|парадный|офицер|офицерский|мундир|мундирный',
    'сельскохозяйственный': r'ферма|фермерский|поле|полевой|урожай|урожайный|трактор|тракторный|агроном|агрономический|зерно|зерновой|овощ|овощной|фрукт|фруктовый|скот|скотский|мелиорация|мелиоративный|техника|технический|посев|посевной|удобрение|удобрительный',
    'медицинский': r'больница|больничный|клиника|клинический|врач|врачебный|хирургия|хирургический|диагностика|диагностический|аптека|аптечный|реабилитация|реабилитационный|стоматология|стоматологический|медцентр|медцентровый|лаборатория|лабораторный|терапия|терапевтический|анализ|аналитический|профилактика|профилактический',
    'IT и технологический': r'стартап|стартаповый|программирование|программный|гаджет|гаджетовый|искусственный интеллект|искусственноинтеллектуальный|робот|роботизированный|кодинг|кодинговый|парк|парковый|инновация|инновационный|криптовалюта|криптовалютный|кибербезопасность|кибербезопасный|хакатон|хакатонный|алгоритм|алгоритмический|данные|данный|технология|технологический',
    'экологический': r'заповедник|заповедный|экология|экологический|природа|природный|озеленение|озеленительный|воздух|воздушный|переработка|перерабатывающий|биоразнообразие|биоразнообразный|животное|животный|парк|парковый|развитие|развитый|охрана|охранный|чистота|чистый',
    'семейный отдых': r'площадка|площадочный|парк|парковый|аквапарк|аквапарковый|отель|отельный|анимация|анимационный|лагерь|лагерный|зоопарк|зоопарковый|мультфильм|мультфильмовый|игра|игровой|тур|туровый|развлечение|развлекательный|ребенок|детский|карусель|карусельный',
    'экзотический': r'тропики|тропический|культура|культурный|животное|животный|природа|природный|страна|страновый|место|местный|традиция|традиционный|кухня|кухонный|племя|племенной|ритуал|ритуальный|ландшафт|ландшафтный|необычность|необычный|загадка|загадочный',
    'спортивный': r'стадион|стадионный|комплекс|комплексный|арена|ареновый|чемпионат|чемпионский|соревнование|соревновательный|олимпиада|олимпийский|футбол|футбольный|баскетбол|баскетбольный|хоккей|хоккейный|теннис|теннисный|атлетика|атлетический|секция|секционный|база|базовый'
}
COMPILED_KEYWORDS = {
    key: re.compile(pattern, re.IGNORECASE)
    for key, pattern in type_keywords.items()
}

def extract_city_type(text, top_n=3):
    text = text[:3000]
    text = lemmatize_text(text)
    text_length = len(text.split())
    scores = []

    for type_name, pattern in COMPILED_KEYWORDS.items():
        matches = pattern.findall(text)
        if matches:
            norm_score = len(matches) / max(1, text_length / 1000)
            scores.append((type_name, norm_score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return [type_name for type_name, _ in scores[:top_n]] if scores else ['unknown']


def get_city_features(city_name):
    city_name = '-'.join([part.capitalize() for part in city_name.lower().split('-')])
    wiki = wikipediaapi.Wikipedia(user_agent='datasearch', language='ru')
    search_variants = [
        city_name + " (город)",  
        city_name,               
        city_name.replace("-", " "),  
        city_name.replace("-", ""),   
    ]
    for city_name in search_variants:
        page = wiki.page(city_name)
        if page.exists():
            print(city_name)
            return extract_city_type(page.text)
    return ['unknown']

In [15]:
df = pd.read_excel('city.xlsx')
df = df[df['city'].notna()]
df = df.drop_duplicates(subset=['city'], keep='first')
df.shape

(1095, 7)

In [16]:
df['features'] = df['city'].apply(get_city_features)

Адыгейск
Майкоп
Горно-Алтайск
Алейск
Барнаул
Белокуриха
Бийск
Горняк (город)
Заринск
Змеиногорск
Новоалтайск
Рубцовск
Славгород
Яровое
Белогорск
Благовещенск (город)
Завитинск
Зея (город)
Райчихинск
Свободный (город)
Сковородино
Тында (город)
Циолковский (город)
Шимановск
Архангельск
Вельск
Каргополь
Коряжма
Котлас
Мезень (город)
Мирный (город)
Новодвинск
Няндома
Онега (город)
Северодвинск
Сольвычегодск
Шенкурск
Астрахань
Ахтубинск
Знаменск (город)
Камызяк (город)
Нариманов (город)
Харабали
Агидель (город)
Баймак (город)
Белебей (город)
Белорецк
Бирск
Давлеканово
Дюртюли
Ишимбай (город)
Кумертау
Межгорье (город)
Мелеуз
Нефтекамск
Октябрьский (город)
Салават (город)
Сибай (город)
Стерлитамак
Туймазы (город)
Уфа (город)
Учалы
Янаул
Алексеевка
Белгород
Бирюч (город)
Валуйки
Грайворон
Губкин (город)
Короча
Строитель (город)
Шебекино
Брянск
Дятьково
Жуковка (город)
Злынка
Карачев
Клинцы
Мглин
Новозыбков
Почеп
Севск
Сельцо (город)
Стародуб
Сураж
Трубчевск
Унеча
Фокино
Бабушкин (город)
Гусино

In [17]:
df.head()

,city,federal_district,region,fias_level,capital_marker,population,foundation_year,features
0,адыгейск,Южный,Адыгея,4,0,12689,1969,"[семейный отдых, курортный, экологический]"
1,майкоп,Южный,Адыгея,4,2,144055,1857,"[семейный отдых, экологический, исторический]"
2,горно-алтайск,Сибирский,Алтай,4,2,62861,1830,"[семейный отдых, исторический, экзотический]"
3,алейск,Сибирский,Алтайский,4,0,28528,1913,"[промышленный, семейный отдых, экологический]"
4,барнаул,Сибирский,Алтайский,4,2,635585,1730,"[промышленный, экзотический, исторический]"


In [18]:
df.to_excel('ready_cities.xlsx')